In [1]:
import numpy as np
import pandas as pd

In [2]:
CATEGORIES = ['Iris-virginica', 'Iris-versicolor']
L = 0.02
EPOCHS = 80
EPS = 0.01
THETA = np.zeros((5,1))

In [3]:
def normalise(dataset):
    return (dataset-dataset.mean())/dataset.std()

In [4]:
def read_file(file, excel=False):
    if excel:
        data = pd.read_excel(file)
    else:
        data = pd.read_csv(file)

    data.rename(columns={'4': 'class'}, inplace=True)
    data.replace(CATEGORIES[0], 0, inplace=True)
    data.replace(CATEGORIES[1], 1, inplace=True)
    x_data = normalise(data[data.columns[:-1]])
    m = x_data.shape[0]
    x_data = pd.DataFrame(np.hstack([np.ones((m,1)), x_data]))
    y_data = data[data.columns[-1]]
    return x_data, y_data

# if __name__ == "__main__":

x_train, y_train = read_file('train_2.xlsx', excel=True)    
x_test, y_test = read_file('test_2.csv')


In [5]:
def sigmoid(x, theta):
    z = np.dot(x, theta)
    return 1/(1+np.exp(-z))

In [6]:
def cost(y, y_predicted):
    cost = -1 * np.sum(y * np.log(y_predicted) + (1 - y) * np.log(1 - y_predicted))
    return cost

In [7]:
def gradient(x,y,y_predicted):
    return np.dot(x.T, (y_predicted - y))

In [8]:
def accuracy(y_predicted, labels):
    y_predicted = (y_predicted > 0.5).astype(int)
    labels = np.reshape(labels, y_predicted.shape)
    assert y_predicted.shape == labels.shape
    return np.mean(y_predicted == labels)

In [9]:
def logistic_regression_epochs(x, y):
    global THETA
    y = y[..., np.newaxis]
    costs = []
    accuracies = []
    for ep in range(EPOCHS):
        y_predicted = sigmoid(x,THETA)
        loss = cost(y, y_predicted)
        grad = gradient(x,y,y_predicted)
        THETA -= L * grad
        costs.append(loss)
        acc = accuracy(y_predicted, y)
        accuracies.append(acc)
        print(f'Epoch : {ep}/epochs\t   Loss : {"%.6f"%loss}\t   Accuracy : {"%.15f"%acc}')
    return accuracies[-1]


def logistic_regression_epsilon(x, y):
    global THETA
    y = y[..., np.newaxis]
    costs = []
    j = 1
    ep = 0
    accuracies = []
    while j > EPS:
        y_predicted = sigmoid(x,THETA)
        loss = cost(y, y_predicted)
        grad = gradient(x,y,y_predicted)
        THETA -= L * grad
        costs.append(loss)
        acc = accuracy(y_predicted, y)
        accuracies.append(acc)
        j = np.linalg.norm(grad)
        print(f'Epoch : {ep}/epochs\t  ||delta J||: {"%.6f"%j}   \tLoss : {"%.6f"%loss}   \tAccuracy : {"%.13f"%acc}')
        ep+=1
    return ep, accuracies[-1]


def compute(x, y):
    global THETA, L
    y_predicted = sigmoid(x,THETA)
    loss = cost(y, y_predicted)
    grad = gradient(x,y,y_predicted)
    THETA -= L * grad
    return loss, grad, y_predicted

def logistic_regression(x, y, eps=False):
    global THETA
    y = y[..., np.newaxis]
    costs = []
    accuracies = []
    if not eps:
        for ep in range(EPOCHS):
            loss, grad, y_predicted = compute(x,y)
            costs.append(loss)
            acc = accuracy(y_predicted, y)
            accuracies.append(acc)
            print(f'Epoch : {ep}/epochs\t   Loss : {"%.6f"%loss}\t   Accuracy : {"%.15f"%acc}')
        return EPOCHS, accuracies[-1]

    else:
        j = 1
        ep = 0
        while j > EPS:
            loss, grad, y_predicted = compute(x,y)
            costs.append(loss)
            acc = accuracy(y_predicted, y)
            accuracies.append(acc)
            j = np.linalg.norm(grad)
            print(f'Epoch : {ep}/epochs\t  ||delta J||: {"%.6f"%j}   \tLoss : {"%.6f"%loss}   \tAccuracy : {"%.13f"%acc}')
            ep+=1
        return ep, accuracies[-1]

        
def evaluate_algorithm(eps=False):
    ep, train_accuracy = logistic_regression(x_train.values,y_train.values, eps)    
    print('------------------------------------------------------------------------------------------------')
    print(f'\nTraining Set Accuracy for {ep} epochs:\t{train_accuracy*100}%')
    test_predictions = sigmoid(x_test.values, THETA)
    test_accuracy = accuracy(test_predictions, y_test.values)
    print(f'Test Set Accuracy:\t{test_accuracy*100}%')
    print('------------------------------------------------------------------------------------------------')

In [10]:
evaluate_algorithm()

Epoch : 0/epochs	   Loss : 49.213450	   Accuracy : 0.549295774647887
Epoch : 1/epochs	   Loss : 27.368299	   Accuracy : 0.859154929577465
Epoch : 2/epochs	   Loss : 23.354341	   Accuracy : 0.915492957746479
Epoch : 3/epochs	   Loss : 20.834552	   Accuracy : 0.929577464788732
Epoch : 4/epochs	   Loss : 19.014674	   Accuracy : 0.943661971830986
Epoch : 5/epochs	   Loss : 17.616352	   Accuracy : 0.943661971830986
Epoch : 6/epochs	   Loss : 16.501924	   Accuracy : 0.929577464788732
Epoch : 7/epochs	   Loss : 15.590847	   Accuracy : 0.929577464788732
Epoch : 8/epochs	   Loss : 14.831320	   Accuracy : 0.929577464788732
Epoch : 9/epochs	   Loss : 14.188004	   Accuracy : 0.943661971830986
Epoch : 10/epochs	   Loss : 13.635805	   Accuracy : 0.943661971830986
Epoch : 11/epochs	   Loss : 13.156359	   Accuracy : 0.943661971830986
Epoch : 12/epochs	   Loss : 12.735915	   Accuracy : 0.943661971830986
Epoch : 13/epochs	   Loss : 12.363970	   Accuracy : 0.943661971830986
Epoch : 14/epochs	   Loss : 12

In [11]:
THETA = np.zeros((5,1))
evaluate_algorithm(eps=True)

Epoch : 0/epochs	  ||delta J||: 42.423349   	Loss : 49.213450   	Accuracy : 0.5492957746479
Epoch : 1/epochs	  ||delta J||: 15.122985   	Loss : 27.368299   	Accuracy : 0.8591549295775
Epoch : 2/epochs	  ||delta J||: 11.719643   	Loss : 23.354341   	Accuracy : 0.9154929577465
Epoch : 3/epochs	  ||delta J||: 9.869763   	Loss : 20.834552   	Accuracy : 0.9295774647887
Epoch : 4/epochs	  ||delta J||: 8.607763   	Loss : 19.014674   	Accuracy : 0.9436619718310
Epoch : 5/epochs	  ||delta J||: 7.658428   	Loss : 17.616352   	Accuracy : 0.9436619718310
Epoch : 6/epochs	  ||delta J||: 6.906955   	Loss : 16.501924   	Accuracy : 0.9295774647887
Epoch : 7/epochs	  ||delta J||: 6.293537   	Loss : 15.590847   	Accuracy : 0.9295774647887
Epoch : 8/epochs	  ||delta J||: 5.782229   	Loss : 14.831320   	Accuracy : 0.9295774647887
Epoch : 9/epochs	  ||delta J||: 5.349318   	Loss : 14.188004   	Accuracy : 0.9436619718310
Epoch : 10/epochs	  ||delta J||: 4.978170   	Loss : 13.635805   	Accuracy : 0.943661971

Epoch : 183/epochs	  ||delta J||: 0.479913   	Loss : 6.251828   	Accuracy : 0.9718309859155
Epoch : 184/epochs	  ||delta J||: 0.477595   	Loss : 6.247233   	Accuracy : 0.9718309859155
Epoch : 185/epochs	  ||delta J||: 0.475299   	Loss : 6.242682   	Accuracy : 0.9718309859155
Epoch : 186/epochs	  ||delta J||: 0.473025   	Loss : 6.238174   	Accuracy : 0.9718309859155
Epoch : 187/epochs	  ||delta J||: 0.470773   	Loss : 6.233710   	Accuracy : 0.9718309859155
Epoch : 188/epochs	  ||delta J||: 0.468542   	Loss : 6.229288   	Accuracy : 0.9718309859155
Epoch : 189/epochs	  ||delta J||: 0.466331   	Loss : 6.224908   	Accuracy : 0.9718309859155
Epoch : 190/epochs	  ||delta J||: 0.464142   	Loss : 6.220569   	Accuracy : 0.9718309859155
Epoch : 191/epochs	  ||delta J||: 0.461972   	Loss : 6.216270   	Accuracy : 0.9718309859155
Epoch : 192/epochs	  ||delta J||: 0.459823   	Loss : 6.212012   	Accuracy : 0.9718309859155
Epoch : 193/epochs	  ||delta J||: 0.457694   	Loss : 6.207793   	Accuracy : 0.97

Epoch : 441/epochs	  ||delta J||: 0.208415   	Loss : 5.727779   	Accuracy : 0.9718309859155
Epoch : 442/epochs	  ||delta J||: 0.207930   	Loss : 5.726911   	Accuracy : 0.9718309859155
Epoch : 443/epochs	  ||delta J||: 0.207448   	Loss : 5.726048   	Accuracy : 0.9718309859155
Epoch : 444/epochs	  ||delta J||: 0.206967   	Loss : 5.725188   	Accuracy : 0.9718309859155
Epoch : 445/epochs	  ||delta J||: 0.206488   	Loss : 5.724332   	Accuracy : 0.9718309859155
Epoch : 446/epochs	  ||delta J||: 0.206012   	Loss : 5.723481   	Accuracy : 0.9718309859155
Epoch : 447/epochs	  ||delta J||: 0.205537   	Loss : 5.722633   	Accuracy : 0.9718309859155
Epoch : 448/epochs	  ||delta J||: 0.205064   	Loss : 5.721789   	Accuracy : 0.9718309859155
Epoch : 449/epochs	  ||delta J||: 0.204593   	Loss : 5.720949   	Accuracy : 0.9718309859155
Epoch : 450/epochs	  ||delta J||: 0.204124   	Loss : 5.720113   	Accuracy : 0.9718309859155
Epoch : 451/epochs	  ||delta J||: 0.203656   	Loss : 5.719280   	Accuracy : 0.97

Epoch : 679/epochs	  ||delta J||: 0.130323   	Loss : 5.596753   	Accuracy : 0.9718309859155
Epoch : 680/epochs	  ||delta J||: 0.130100   	Loss : 5.596413   	Accuracy : 0.9718309859155
Epoch : 681/epochs	  ||delta J||: 0.129879   	Loss : 5.596075   	Accuracy : 0.9718309859155
Epoch : 682/epochs	  ||delta J||: 0.129658   	Loss : 5.595738   	Accuracy : 0.9718309859155
Epoch : 683/epochs	  ||delta J||: 0.129437   	Loss : 5.595402   	Accuracy : 0.9718309859155
Epoch : 684/epochs	  ||delta J||: 0.129217   	Loss : 5.595067   	Accuracy : 0.9718309859155
Epoch : 685/epochs	  ||delta J||: 0.128998   	Loss : 5.594734   	Accuracy : 0.9718309859155
Epoch : 686/epochs	  ||delta J||: 0.128779   	Loss : 5.594401   	Accuracy : 0.9718309859155
Epoch : 687/epochs	  ||delta J||: 0.128561   	Loss : 5.594070   	Accuracy : 0.9718309859155
Epoch : 688/epochs	  ||delta J||: 0.128343   	Loss : 5.593739   	Accuracy : 0.9718309859155
Epoch : 689/epochs	  ||delta J||: 0.128126   	Loss : 5.593410   	Accuracy : 0.97

Epoch : 944/epochs	  ||delta J||: 0.086778   	Loss : 5.535927   	Accuracy : 0.9718309859155
Epoch : 945/epochs	  ||delta J||: 0.086657   	Loss : 5.535777   	Accuracy : 0.9718309859155
Epoch : 946/epochs	  ||delta J||: 0.086536   	Loss : 5.535627   	Accuracy : 0.9718309859155
Epoch : 947/epochs	  ||delta J||: 0.086415   	Loss : 5.535477   	Accuracy : 0.9718309859155
Epoch : 948/epochs	  ||delta J||: 0.086294   	Loss : 5.535328   	Accuracy : 0.9718309859155
Epoch : 949/epochs	  ||delta J||: 0.086174   	Loss : 5.535179   	Accuracy : 0.9718309859155
Epoch : 950/epochs	  ||delta J||: 0.086054   	Loss : 5.535031   	Accuracy : 0.9718309859155
Epoch : 951/epochs	  ||delta J||: 0.085935   	Loss : 5.534883   	Accuracy : 0.9718309859155
Epoch : 952/epochs	  ||delta J||: 0.085815   	Loss : 5.534735   	Accuracy : 0.9718309859155
Epoch : 953/epochs	  ||delta J||: 0.085696   	Loss : 5.534588   	Accuracy : 0.9718309859155
Epoch : 954/epochs	  ||delta J||: 0.085577   	Loss : 5.534441   	Accuracy : 0.97

...

Epoch : 2145/epochs	  ||delta J||: 0.022065   	Loss : 5.478297   	Accuracy : 0.9718309859155
Epoch : 2146/epochs	  ||delta J||: 0.022043   	Loss : 5.478287   	Accuracy : 0.9718309859155
Epoch : 2147/epochs	  ||delta J||: 0.022021   	Loss : 5.478277   	Accuracy : 0.9718309859155
Epoch : 2148/epochs	  ||delta J||: 0.021999   	Loss : 5.478268   	Accuracy : 0.9718309859155
Epoch : 2149/epochs	  ||delta J||: 0.021977   	Loss : 5.478258   	Accuracy : 0.9718309859155
Epoch : 2150/epochs	  ||delta J||: 0.021955   	Loss : 5.478248   	Accuracy : 0.9718309859155
Epoch : 2151/epochs	  ||delta J||: 0.021933   	Loss : 5.478239   	Accuracy : 0.9718309859155
Epoch : 2152/epochs	  ||delta J||: 0.021911   	Loss : 5.478229   	Accuracy : 0.9718309859155
Epoch : 2153/epochs	  ||delta J||: 0.021889   	Loss : 5.478219   	Accuracy : 0.9718309859155
Epoch : 2154/epochs	  ||delta J||: 0.021867   	Loss : 5.478210   	Accuracy : 0.9718309859155
Epoch : 2155/epochs	  ||delta J||: 0.021845   	Loss : 5.478200   	Accu

Epoch : 2359/epochs	  ||delta J||: 0.017846   	Loss : 5.476599   	Accuracy : 0.9718309859155
Epoch : 2360/epochs	  ||delta J||: 0.017828   	Loss : 5.476593   	Accuracy : 0.9718309859155
Epoch : 2361/epochs	  ||delta J||: 0.017811   	Loss : 5.476587   	Accuracy : 0.9718309859155
Epoch : 2362/epochs	  ||delta J||: 0.017793   	Loss : 5.476580   	Accuracy : 0.9718309859155
Epoch : 2363/epochs	  ||delta J||: 0.017776   	Loss : 5.476574   	Accuracy : 0.9718309859155
Epoch : 2364/epochs	  ||delta J||: 0.017759   	Loss : 5.476568   	Accuracy : 0.9718309859155
Epoch : 2365/epochs	  ||delta J||: 0.017741   	Loss : 5.476561   	Accuracy : 0.9718309859155
Epoch : 2366/epochs	  ||delta J||: 0.017724   	Loss : 5.476555   	Accuracy : 0.9718309859155
Epoch : 2367/epochs	  ||delta J||: 0.017706   	Loss : 5.476549   	Accuracy : 0.9718309859155
Epoch : 2368/epochs	  ||delta J||: 0.017689   	Loss : 5.476542   	Accuracy : 0.9718309859155
Epoch : 2369/epochs	  ||delta J||: 0.017672   	Loss : 5.476536   	Accu

Epoch : 2516/epochs	  ||delta J||: 0.015316   	Loss : 5.475738   	Accuracy : 0.9718309859155
Epoch : 2517/epochs	  ||delta J||: 0.015301   	Loss : 5.475733   	Accuracy : 0.9718309859155
Epoch : 2518/epochs	  ||delta J||: 0.015286   	Loss : 5.475728   	Accuracy : 0.9718309859155
Epoch : 2519/epochs	  ||delta J||: 0.015271   	Loss : 5.475724   	Accuracy : 0.9718309859155
Epoch : 2520/epochs	  ||delta J||: 0.015257   	Loss : 5.475719   	Accuracy : 0.9718309859155
Epoch : 2521/epochs	  ||delta J||: 0.015242   	Loss : 5.475714   	Accuracy : 0.9718309859155
Epoch : 2522/epochs	  ||delta J||: 0.015227   	Loss : 5.475710   	Accuracy : 0.9718309859155
Epoch : 2523/epochs	  ||delta J||: 0.015213   	Loss : 5.475705   	Accuracy : 0.9718309859155
Epoch : 2524/epochs	  ||delta J||: 0.015198   	Loss : 5.475700   	Accuracy : 0.9718309859155
Epoch : 2525/epochs	  ||delta J||: 0.015183   	Loss : 5.475696   	Accuracy : 0.9718309859155
Epoch : 2526/epochs	  ||delta J||: 0.015168   	Loss : 5.475691   	Accu

Epoch : 2728/epochs	  ||delta J||: 0.012497   	Loss : 5.474920   	Accuracy : 0.9718309859155
Epoch : 2729/epochs	  ||delta J||: 0.012485   	Loss : 5.474917   	Accuracy : 0.9718309859155
Epoch : 2730/epochs	  ||delta J||: 0.012473   	Loss : 5.474914   	Accuracy : 0.9718309859155
Epoch : 2731/epochs	  ||delta J||: 0.012461   	Loss : 5.474911   	Accuracy : 0.9718309859155
Epoch : 2732/epochs	  ||delta J||: 0.012450   	Loss : 5.474908   	Accuracy : 0.9718309859155
Epoch : 2733/epochs	  ||delta J||: 0.012438   	Loss : 5.474905   	Accuracy : 0.9718309859155
Epoch : 2734/epochs	  ||delta J||: 0.012426   	Loss : 5.474902   	Accuracy : 0.9718309859155
Epoch : 2735/epochs	  ||delta J||: 0.012414   	Loss : 5.474899   	Accuracy : 0.9718309859155
Epoch : 2736/epochs	  ||delta J||: 0.012402   	Loss : 5.474896   	Accuracy : 0.9718309859155
Epoch : 2737/epochs	  ||delta J||: 0.012390   	Loss : 5.474893   	Accuracy : 0.9718309859155
Epoch : 2738/epochs	  ||delta J||: 0.012379   	Loss : 5.474889   	Accu

Epoch : 2889/epochs	  ||delta J||: 0.010729   	Loss : 5.474487   	Accuracy : 0.9718309859155
Epoch : 2890/epochs	  ||delta J||: 0.010719   	Loss : 5.474485   	Accuracy : 0.9718309859155
Epoch : 2891/epochs	  ||delta J||: 0.010709   	Loss : 5.474482   	Accuracy : 0.9718309859155
Epoch : 2892/epochs	  ||delta J||: 0.010699   	Loss : 5.474480   	Accuracy : 0.9718309859155
Epoch : 2893/epochs	  ||delta J||: 0.010689   	Loss : 5.474478   	Accuracy : 0.9718309859155
Epoch : 2894/epochs	  ||delta J||: 0.010679   	Loss : 5.474475   	Accuracy : 0.9718309859155
Epoch : 2895/epochs	  ||delta J||: 0.010669   	Loss : 5.474473   	Accuracy : 0.9718309859155
Epoch : 2896/epochs	  ||delta J||: 0.010659   	Loss : 5.474471   	Accuracy : 0.9718309859155
Epoch : 2897/epochs	  ||delta J||: 0.010648   	Loss : 5.474469   	Accuracy : 0.9718309859155
Epoch : 2898/epochs	  ||delta J||: 0.010638   	Loss : 5.474466   	Accuracy : 0.9718309859155
Epoch : 2899/epochs	  ||delta J||: 0.010628   	Loss : 5.474464   	Accu